## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool

#OpenAI SDK data model. Used when streaming responses. Displays to the user while being typed, instead of waiting for response.
from openai.types.responses import ResponseTextDeltaEvent 

#used for type hinting 
from typing import Dict

import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

import os
import asyncio

#libraries to fix certificate error
import ssl
import certifi


In [ ]:
load_dotenv(override=True)

#this fixes certificate error 
os.environ['SSL_CERT_FILE'] = certifi.where()
ssl_context = ssl.create_default_context(cafile=certifi.where())

In [12]:
# Let's just check emails are working for you
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("gabrielle.carpenter.25@ucl.ac.uk")  # Change to your verified sender
    to_email = To("gabrielle.carpenter.25@ucl.ac.uk")  # Change to your recipient
    content = Content("text/plain", "This is an important test email") #/plain is raw text, no styling; /html allows html styling specs eg. <b>
    mail = Mail(from_email, to_email, "Test email", content).get() #.get() converts into standard python dictionary to be turned into JSON
    response = sg.client.mail.send.post(request_body=mail) #.post is an HTTP POST request -- pushes mail dictionary to send grid
    print(response.status_code)

send_test_email()

202


### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [14]:
#instructions/system prompts to set up context of three different agents

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [15]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:
#Runner carries out the work and uses a tool if called for
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email") #.run_streamed streams the text instead of waiting for a response

#.stream_events() manages steam of events (eg. tool call, handoffs, tool result, text)
#everytime a new event arrives, run the code inside this loop
async for event in result.stream_events(): 
    #event.type == "raw_response_event" - looks only at events that looks at the response of the LLM
    #isinstance(event.data, ReponseDeltaEvent) - from those events, make sure it is a ResponseTextDeltaEvent object (an actual string of text)
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        #.data.delta prints the data that is text (.delta)
        #end='' makes sure that the words stay side-by-side
        print(event.data.delta, end="", flush=True) 

Subject: Simplify Your SOC 2 Compliance Journey with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. 

Navigating the complexities of SOC 2 compliance can be daunting, especially as the stakes continue to rise with regulatory scrutiny. At ComplAI, we understand the challenges you face in managing compliance efficiently while still focusing on your core business objectives.

Our AI-driven SaaS tool streamlines the compliance process, helping organizations like yours prepare for audits with ease. With features designed to automate documentation, monitor controls, and generate comprehensive reports, we empower teams to reduce the time and resources spent on compliance efforts.

By implementing ComplAI, you can:

- **Save Time**: Automate repetitive tasks and focus on strategic initiatives.
- **Enhance Accuracy**: Leverage AI to ensure your documentation is always up-to-date.
- **Facilitate Audits**: Access organized reports and evidence at your fingertips.

I would lo

In [ ]:
message = "Write a cold sales email"

with trace("Parallel cold emails"): #inside names the trace for easier monitoring
    results = await asyncio.gather( #parallelizes the three agents          
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

#list called 'outputs'
#.final_output grabs just the final output of the results (the output of each sales agent)
#for result in results tells python to loop through existing results list
outputs = [result.final_output for result in results]

#prints each result.final_output (or "output")
for output in outputs:
    print(output + "\n\n")


Subject: Streamline Your SOC 2 Compliance Efforts with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out to introduce you to ComplAI, a cutting-edge SaaS platform designed to simplify the process of achieving and maintaining SOC 2 compliance.

In today’s rapidly evolving regulatory landscape, ensuring compliance can become a daunting task. ComplAI leverages AI-driven insights to help organizations like yours seamlessly navigate the complexities of SOC 2 requirements, saving you time and reducing the risk of non-compliance.

Here’s how ComplAI can add value to your compliance efforts:

- **Automated Compliance Management**: Minimize manual processes and reduce human error with our automation features.
- **Real-Time Monitoring**: Stay ahead of compliance requirements with proactive alerts and updates tailored to your organization’s needs.
- **Audit Preparedness**: Easily compile necessary documentation and reports, ensuring 

In [18]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [19]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    #emails becomes the outputs, separated by the headers 'cold sales emails:' and 'email:'
    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs) 

    #await allows agent to switch between agents if they're working at different speeds
    #passing sales_picker and emails through
    #Runner.run takes the following arguments: Runner.run(agent, input, context[this is optional], model[also optional])
    best = await Runner.run(sales_picker, emails) 

    #prints the best sales email - the final output of var best
    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Is Your SOC2 Compliance Giving You Nightmares? 😴💼

Hey [First Name]!

Ever wake up in a cold sweat, dreaming about endless spreadsheets and looming audit deadlines? Yeah, me too! 😅

At ComplAI, we’ve got your back! Our AI-powered tool is like a superhero for your compliance needs—think of it as the cape that makes SOC2 compliance feel less like wrestling a bear and more like having a charming tea party. 🦸‍♂️🫖

Here’s what we bring to the table:
- **Seamless Compliance Tracking:** No more hiding in the corner hoping it all magically falls into place!
- **Effortless Audit Prep:** Wave goodbye to the days of scrambling for paperwork like it’s a game of hide-and-seek.
- **24/7 Support:** Like a trusty sidekick, we’re always here for you—minus the leapfrogging.

Let’s chat about how we can transform your compliance headaches into sweet dreams! If you have a few minutes this week, I’d love to schedule a quick call. Just hit "reply" or grab some time on my calendar 

Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [20]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [21]:
sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [22]:
#bridge that turns a regular python function to something an AI Agent can understand/use
#eliminates the need to manually write out a python dictionary and then transform that into JSON
#enables the AI to see the actual function and encode that into a tool for an agent

@function_tool 
def send_email(body: str): #body: str is a type hint that tells programmer and AI what kind of data function expects to recieve - a block of text (not JSON object or list of sentences, for example)
    """ Send out an email with the given body to all sales prospects """ #creates a manual for the AI -- AI knows when to call it

    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("gabrielle.carpenter.25@ucl.ac.uk")  # Change to your verified sender
    to_email = To("gabrielle.carpenter.25@ucl.ac.uk")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [23]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000200EE61D9E0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

### And you can also convert an Agent into a tool

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000200EE089800>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [ ]:
description = "Write a cold sales email" 
#argument tool_description gives a label to the tool so LLM can scan all the tools and choose them to use if needed

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email] #now we have a complete list of tools

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000200EE088A40>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000200EE0884A0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', descr

## And now it's time for our Sales Manager - our planning agent

In [26]:
# Sales agent that can use the four tools specified: three different sales agent and the send email tool

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [29]:
#two agents: one that writes subject for an email, and one that converts a text email body to an HTML email body

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [30]:
@function_tool

#type hint tells the result of this function (a dictionary with two values that are in string form)
#the two parameters signify that the AI must fill both requirements before sending the email
def send_html_email(subject: str, html_body: str) -> Dict[str, str]: 
    """ Send out an email with the given subject and HTML body to all sales prospects """

    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("gabrielle.carpenter.25@ucl.ac.uk")  # Change to your verified sender
    to_email = To("gabrielle.carpenter.25@ucl.ac.uk")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [31]:
tools = [subject_tool, html_tool, send_html_email]

In [32]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000200EDDB0540>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000200EDDB3BA0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=No

In [34]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it" #this clarifies that this agent is a handoff tool. LLM will choose this if seen fit
)

### Now we have 3 tools and 1 handoff

In [35]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000200EE088A40>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000200EE0884A0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent3', descrip

In [36]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.